#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2018


# Homework 3:  Embeddings + Recommenders

### 100 points [5% of your final grade]

### Due: Monday, April 9 by 11:59pm

*Goals of this homework:* There are two main learning objectives: (i) implement and evaluate a pre-cursor to modern word2vec embeddings; and (ii) implement, evaluate, and improve upon traditional collaborative filtering recommenders.

*Submission Instructions:* To submit your homework, rename this notebook as UIN_hw#.ipynb. For example, this homework submission would be: YourUIN_hw3.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Thursday, April 12 at 11:59pm.

# Part 1: Word Embeddings (50 points)
For this first part, we're going to implement a word embedding approach that is a bit simpler than word2vec. The key idea is to look at co-occurrences between center words and context words (somewhat like in word2vec) but without any pesky learning of model parameters.

If you're interested in a deeper treatment of comparing count vs. learned embeddings, take a look at: [Don’t count, predict! A systematic comparison of
context-counting vs. context-predicting semantic vectors](
http://www.aclweb.org/anthology/P14-1023)

## Load the Brown Corpus

The dataset for this part is the (in)famous [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) that is a collection of text samples from a wide range of sources, with over one million unique words. Good for us, you can find the Brown corpus in nltk. *Make sure you have already installed nltk with something like: conda install nltk*

In [1]:
import nltk
nltk.download('brown')
nltk.download('stopwords')

[nltk_data] Downloading package brown to /Users/Apple/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/Apple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Once you have it locally, you can load the dataset into your notebook. You can access the words using brown.words():

In [2]:
from nltk.corpus import brown
# sents = brown.sents()
# sents = list(sents)
# sents = [["jack marcus done jack marcus jack jack"]]
print(brown.words())
bwords = list(brown.words())

[u'The', u'Fulton', u'County', u'Grand', u'Jury', ...]


## 1.1 Dataset Pre-processing
OK, now we need to do some basic pre-processing. For this part you should:

* Remove stopwords and punctuation.
* Make everything lowercase.

Then, count how often each word occurs. We will define the 5,000 most  frequent words as your vocabulary (V). We will define the 1,000 most frequent words as our context (C). Include a print statement below to show the top-20 words after pre-processing.

In [3]:
import string
import nltk
from nltk.corpus import stopwords

words = []
for w in bwords:
    words.append(str(w).translate(None, string.punctuation).lower())
words = [w for w in words if w not in stopwords.words('english') and len(w)>0]

In [5]:
from collections import Counter
bag = Counter()
bag.update(words)
V = [x[0] for x in bag.most_common(5000)]
C = [x[0] for x in bag.most_common(1000)]
print(bag.most_common(20))

[('one', 3297), ('would', 2714), ('said', 1961), ('new', 1635), ('could', 1601), ('time', 1598), ('two', 1412), ('may', 1402), ('first', 1361), ('like', 1292), ('man', 1207), ('even', 1170), ('made', 1125), ('also', 1069), ('many', 1030), ('must', 1013), ('years', 1001), ('af', 996), ('back', 966), ('well', 961)]


## 1.2 Building the Co-occurrence Matrix 

For each word in the vocabulary (w), we want to calculate how often context words from C appear in its surrounding window of size 4 (two words before and two words after).

In other words, we need to define a co-occurrence matrix that has a dimension of |V|x|C| such that each cell (w,c) represents the number of times c occurs in a window around w. 

In [6]:
from nltk import ngrams
import numpy as np

vocab_to_index = {word:i for i, word in enumerate(V)}
index_to_vocab = {i:word for i, word in enumerate(V)}
context_to_index = {word:i for i, word in enumerate(C)}
co_occurrence_matrix = np.zeros((len(V), len(C)))

n = 5
fivegrams = ngrams(words, n)
for gram in fivegrams:
    gram = list(gram)
    central_word = gram[2]
    if (central_word not in V):
        continue;
    for i in np.arange(5):
        if (i == 2):
            continue
        if (gram[i] in C):
            co_occurrence_matrix[vocab_to_index[central_word]][context_to_index[gram[i]]] += 1

print(co_occurrence_matrix)

[[ 82.  76.  52. ...,   2.   1.   0.]
 [ 76.  70.  68. ...,   2.   1.   0.]
 [ 52.  68.  34. ...,   4.   1.   0.]
 ..., 
 [  0.   1.   0. ...,   0.   0.   0.]
 [  0.   0.   0. ...,   0.   0.   0.]
 [  0.   0.   0. ...,   0.   0.   0.]]


## 1.3 Probability Distribution

Using the co-occurrence matrix, we can compute the probability distribution Pr(c|w) of context word c around w as well as the overall probability distribution of each context word c with Pr(c).  

In [7]:
com = np.array(co_occurrence_matrix)
pcw = com/com.sum(axis=1, keepdims=True)
pcw[np.isnan(pcw)] = 0     # Making all NaNs to 0
# print(pcw)

In [8]:
pw = [x[1] for x in bag.most_common(1000)]
pw = [float(p)/len(words) for p in pw]
# print pw

## 1.4 Embedding Representation

Now you can represent each vocabulary word as a |C| dimensional vector using this equation:

Vector(w)= max(0, log (Pr(c|w)/Pr(c)))

This is a traditional approach called *pointwise mutual information* that pre-dates word2vec by some time. 

In [9]:
embeddings = np.log(pcw/pw)
b = embeddings < 0
embeddings[b] = 0
print(embeddings)

[[ 0.74703104  0.86563454  0.81111393 ...,  0.55951624  0.          0.        ]
 [ 0.87100003  0.98335134  1.27933281 ...,  0.75947114  0.06632396  0.        ]
 [ 0.84881633  1.31166972  0.94349155 ...,  1.80992424  0.42362987  0.        ]
 ..., 
 [ 0.          1.65541311  0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


## 1.5 Analysis

So now we have some embeddings for each word. But are they meaningful? For this part, you should:

- First, cluster the vocabulary into 100 clusters using k-means. Look over the words in each cluster, can you see any relation beween words? Discuss your observations.

- Second, for the top-20 most frequent words, find the nearest neighbors using cosine distance (1- cosine similarity). Do the findings make sense? Discuss.

In [10]:
# k-means
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=100).fit(embeddings)
clusters = []

def ClusterIndicesNumpy(clustNum, labels_array):
    return np.where(labels_array == clustNum)[0]

for i in range(len(kmeans.labels_)):
    idx = ClusterIndicesNumpy(i,kmeans.labels_)
    clusters.append([index_to_vocab[i] for i in idx])

print(clusters)

[['rain'], ['civilization'], ['played', 'worth', 'develop', 'dress', 'discussed', 'leaves', 'ballet', 'qualities', 'shoes', 'dancing', 'remaining', 'whereas', 'sister', 'blame', 'naked', 'voting', 'dreams', 'magnificent', 'charm', 'woods', 'exhibit', 'torn', 'devil', 'charming', 'enthusiastic', 'promising', 'adults', 'chart', 'obliged', 'reverend', 'acceptable', 'blonde', 'detectives', 'tent', 'puzzled'], ['un', 'india', 'entitled', 'payment', 'co', 'assembly', 'eisenhower', 'request', 'constitution', 'limit', 'adams', 'representative', 'recommended', 'award', 'edward', 'farmers', 'proposal', 'vice', 'dean', 'treasury', 'approved', 'communists', 'provisions', 'code', 'reserve', 'authorized', 'navy', 'assistant', 'jackson', 'atlanta', '1962', 'revenue', 'thereby', '19', 'charter', 'presentation', 'concluded', 'loans', 'sen', 'medicine', 'fees', 'davis', 'sherman', 'regulations', 'jurisdiction', 'guilty', 'argued', 'contributions', 'commander', 'marshall', 'requirement', 'kennedys', 'nat

In [11]:
# nearest neighbors
top_20 = [x[0] for x in bag.most_common(20)]
# print(top_20)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10,metric='cosine')
y = np.arange(embeddings.shape[0])
knn.fit(embeddings, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [12]:
nn_index = knn.kneighbors(embeddings, return_distance=False)

In [13]:
print(nn_index)

[[   0    8   10 ...,   39   42    5]
 [   1    4   42 ...,    2   28   24]
 [   2    9   40 ...,  105   31  231]
 ..., 
 [4997 2028 4984 ...,  201 2856 2432]
 [4998  202  665 ...,  290  307  641]
 [4999 4111  831 ..., 4755   14  321]]


In [14]:
nn = []
for n in nn_index[:20]:
    nn.append([index_to_vocab[n[i]] for i in range(1,10)])
for i in range(len(nn)):
    print(V[i]+"{}".format(nn[i]))

one['first', 'man', 'every', 'would', 'even', 'could', 'another', 'might', 'time']
would['could', 'might', 'must', 'may', 'much', 'one', 'said', 'make', 'us']
said['like', 'know', 'get', 'dont', 'didnt', 'think', 'told', 'see', 'im']
new['also', 'first', 'many', 'may', 'york', 'world', 'work', 'one', 'made']
could['would', 'said', 'might', 'way', 'didnt', 'one', 'man', 'come', 'dont']
time['one', 'way', 'first', 'even', 'would', 'still', 'much', 'could', 'well']
two['three', 'one', 'many', 'four', 'several', 'first', 'found', 'time', 'long']
may['also', 'would', 'must', 'might', 'one', 'made', 'even', 'new', 'however']
first['one', 'last', 'time', 'second', 'two', 'next', 'work', 'every', 'new']
like['said', 'know', 'man', 'little', 'get', 'come', 'way', 'could', 'even']
man['one', 'like', 'could', 'little', 'still', 'never', 'said', 'way', 'people']
even['much', 'one', 'still', 'would', 'like', 'might', 'well', 'time', 'said']
made['make', 'one', 'may', 'also', 'would', 'first', 'must

# Part 2. Collaborative Filtering (50 points)

In this second part, you will implement collaborative filtering on the Netflix prize dataset -- don’t freak out, the provided sample dataset has only ~2000 items and ~28,000 users.

As background, read the paper [Empirical Analysis of Predictive Algorithms for Collaborative Filtering](https://arxiv.org/pdf/1301.7363.pdf) up to Section 2.1. Of course you can read further if you are interested, and you can also refer to the course slides for collaborative filtering.

## 2.1 Load Netflix Data

The dataset is subset of movie ratings data from the Netflix Prize Challenge. Download the dataset from Piazza. It contains a train set, test set, movie file, and README file. The last two files are original ones from the Netflix Prize, however; in this homework you will deal with train and test files which both are subsets of the Netflix training data. Each of train and test files has lines having this format: MovieID,UserID,Rating.

Your job is to predict a rating in the test set using those provided in the training set.

In [15]:
# load the data, then print out the number of ratings, 
# movies and users in each of train and test sets.
# load the data, then print out the number of ratings, 
# movies and users in each of train and test sets.

import pandas as pd
train_df = pd.read_csv('netflix-dataset/TrainingRatings.txt', header = None, usecols=[0,1,2])
# print(train_df[:10])
print("Number of Movies in train set = " + str(train_df.loc[:,0].unique().size))
print("Number of Users in train set = " + str(train_df.loc[:,1].unique().size))
print("Number of Ratings in train set = " + str(train_df.loc[:,2].unique().size))

Number of Movies in train set = 1821
Number of Users in train set = 28978
Number of Ratings in train set = 5


In [16]:
test_df = pd.read_csv('netflix-dataset/TestingRatings.txt', header = None)
# print(test_df[:10])
print("Number of Movies in test set = " + str(test_df.loc[:,0].unique().size))
print("Number of Users in test set = " + str(test_df.loc[:,1].unique().size))
print("Number of Ratings in test set = " + str(test_df.loc[:,2].unique().size))

Number of Movies in test set = 1701
Number of Users in test set = 27555
Number of Ratings in test set = 5


## 2.2 Implement CF

In this part, you will implement the basic collaborative filtering algorithm described in Section 2.1 of the paper -- that is, focus only on Equations 1 and 2 (where Equation 2 is just the Pearson correlation). You should consider the first 5,000 users with their associated items in the test set. 

Note that you should test the algorithm for a small set of users e.g., 10 users first and then run for 5,000 users. It may take long to run but you won't have memory issues. 

Set k to 0.1. 

In [92]:
grouped = train_df.groupby(1)
n = 10
users_data = [g for g in list(grouped)[:n]]

In [101]:
import numpy as np
userid = np.array([u[0] for u in users_data])

user_data = []
for g in users_data:
    movie_ids = g[1][0].values
    ratings = g[1][2].values
    user_data.append(zip(movie_ids,ratings))
    
user_data = np.array(user_data)
user_data[0]

[(8, 5.0),
 (28, 4.0),
 (185, 4.0),
 (636, 4.0),
 (1046, 3.0),
 (1202, 4.0),
 (1289, 1.0),
 (1406, 3.0),
 (1482, 5.0),
 (1832, 4.0),
 (1901, 5.0),
 (2198, 3.0),
 (2290, 4.0),
 (2660, 5.0),
 (2755, 3.0),
 (2757, 3.0),
 (3151, 2.0),
 (3165, 4.0),
 (3285, 4.0),
 (3290, 5.0),
 (3418, 3.0),
 (3538, 5.0),
 (3541, 5.0),
 (3638, 5.0),
 (3740, 3.0),
 (3890, 3.0),
 (3893, 3.0),
 (4432, 5.0),
 (4546, 4.0),
 (4627, 4.0),
 (4640, 5.0),
 (4876, 3.0),
 (4932, 4.0),
 (5069, 4.0),
 (5149, 4.0),
 (5562, 4.0),
 (5607, 4.0),
 (5814, 3.0),
 (6085, 4.0),
 (6287, 5.0),
 (6336, 4.0),
 (6347, 2.0),
 (6408, 5.0),
 (6523, 3.0),
 (6911, 3.0),
 (6971, 4.0),
 (7067, 4.0),
 (7145, 5.0),
 (7511, 2.0),
 (7577, 3.0),
 (7899, 3.0),
 (8384, 3.0),
 (8428, 3.0),
 (8596, 5.0),
 (8677, 3.0),
 (9043, 5.0),
 (9131, 3.0),
 (9481, 5.0),
 (9617, 5.0),
 (9728, 4.0),
 (10082, 3.0),
 (10255, 5.0),
 (10429, 2.0),
 (10712, 3.0),
 (10774, 4.0),
 (10835, 5.0),
 (10921, 3.0),
 (10947, 5.0),
 (11080, 3.0),
 (11182, 5.0),
 (11613, 5.0),
 (

## 2.3 Evaluation 

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [ ]:
# Mean Absolute Error

In [ ]:
# Root Mean Squared Error

## 2.4 Extensions

Given your results in the previous part, can you do better? For this last part you should report on your best attempt at improving MAE and RMSE. Provide code, results, plus a brief discussion on your approach.

In [ ]:
# Your Code Here...

*Insert discussion here*